<a href="https://colab.research.google.com/github/BiancaStadl/ProjektarbeitML/blob/main/CNN_embedded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Quelle für Twitter-Daten: https://github.com/uds-lsv/GermEval-2018-Data/

Michael Wiegand, Melanie Siegel, and Josef Ruppenhofer: "Overview of the GermEval 2018 Shared Task on the Identification of Offensive Language", in Proceedings of the GermEval, 2018, Vienna, Austria.

In [ ]:
#import matplotlib.pyplot as plt -> für evtl Visualisierungen
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import losses 
from keras import optimizers 
from keras import metrics 

#!pip install Tokenizer
from keras.preprocessing.text import Tokenizer
#!pip install pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
#from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


Einige Parameter für das Netz setzen.. 

In [ ]:
training_epochs = 10
batch_size = 60
learning_rate = 0.01
max_length = 60

In [ ]:
url = "https://github.com/uds-lsv/GermEval-2018-Data/archive/master.zip"

dataset = tf.keras.utils.get_file("GermEval-2018-Data-master.zip", url, 
                                   extract=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'GermEval-2018-Data-master')


In [ ]:
os.listdir(dataset_dir)

['survey.pdf',
 'submissions.tgz',
 'LICENSE.md',
 'germeval2018.training.txt',
 'germeval2018.test.txt',
 'evaluationScriptGermeval2018.pl',
 'README.md',
 'results.pdf',
 'guidelines-iggsa-shared.pdf']

In [ ]:
training_file = os.path.join(dataset_dir, 'germeval2018.training.txt')

testing_file = os.path.join(dataset_dir, 'germeval2018.test.txt')


###Aufbereitung
txt-Dateien lesen, und dabei jede Zeile splitten und teilen
Twitter-Handler entfernen?
Labels von Sätzen trennen, Labels in eigenen Array packen.

####Ressourcen
* https://www.w3schools.com/python/gloss_python_if_statement.asp
* https://stackabuse.com/using-regex-for-text-manipulation-in-python/
* https://www.tutorialspoint.com/python/string_splitlines.htm 
* https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
* https://www.w3schools.com/python/python_regex.asp 

gäbe bei keras tf.keras.preprocessing.text_dataset_from_directory, aber dafür müssten die Daten auf bestimmte Weise in Verzeichnis organisiert sein, hier nicht der Fall.


Einteilung der Daten: Testdaten
Trainingsdaten -> noch ca 500 für Validierungsdaten

labels -> "other", also nicht-negative Aussagen bekommen "0", negative Aussagen bekommen "1"

 Regex für Emojis  ->

(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])

genommen von https://www.regextester.com/106421 

In [ ]:
def remove_clutter(string):
   string = re.sub("@[^\s]+"," ",string)
   string = re.sub("#[^\s]+"," ", string)
   string = re.sub("\u00a9"," ", string)
   string = re.sub("\u00ae"," ", string)
   string = re.sub("[\u2000-\u3300]"," ", string)
   string = re.sub("\ud83c[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83d[\ud000-\udfff]"," ", string)
   string = re.sub("\ud83e[\ud000-\udfff]"," ", string)
   string = re.sub("😜", " ",string)
   string = re.sub("🍫", " ",string)
   string = re.sub("😁", " ",string)
   string = re.sub("🐖", " ",string)
   string = re.sub("😡", " ",string)
   string = re.sub("😇", " ",string)
   string = re.sub("😬", " ",string)
   string = re.sub("😃", " ",string)
   string = re.sub("😂", " ",string)
   string = re.sub("💙", " ",string)  
   string = re.sub("😛", " ",string)
   string = re.sub("🙏", " ",string)
   string = re.sub("👍", " ",string)
   string = re.sub("🖕", " ",string)
   string = re.sub("😉", " ",string)
   string = re.sub("💩", " ",string)
   string = re.sub("🤢", " ",string)
   string = re.sub("👏", " ",string)
   string = re.sub("😨", " ",string)
   string = re.sub("🤣", " ",string)
   string = re.sub("🤡", " ",string)
   string = re.sub("😈", " ",string)
   string = re.sub("💃🏽", " ",string)
   string = re.sub("👹", " ",string)
   string = re.sub("🤘", " ",string)
   string = re.sub("😱", " ",string)
   string = re.sub("🤔", " ",string) 
   string = re.sub("🌈", " ",string) 
   string = re.sub("💕", " ",string) 
   string = re.sub("👩‍❤️‍👩", " ",string) 
   string = re.sub("😍", " ",string) 
   string = re.sub("👆", " ",string) 
   string = re.sub("😖", " ",string) 
   string = re.sub("👇", " ",string) 
   string = re.sub("🔥", " ",string) 
   string = re.sub("😘", " ",string) 
   string = re.sub("🎉", " ",string) 
   string = re.sub("🤬", " ",string) 
   string = re.sub("👊", " ",string)
   string = re.sub("🇩🇪", " ",string)  
   
   string = re.sub("OTHER|OFFENSE|ABUSE|INSULT"," ",string)
   return string

In [ ]:
statementsForTraining = []
sentimentsForTraining = []


fileToRead = open(training_file, 'r')

while True:
  #next line in file
  line = fileToRead.readline()

  if line == "":
   break

  findSentiment = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)


  statementsForTraining.append(line)
   #sentimentsForTraining.append(findSentiment.group(0))

  if findSentiment.group(0) == "OTHER":  
    sentimentsForTraining.append(0)
  else:
    sentimentsForTraining.append(1)

  if not line:
    break

 #print("{}: {}".format(count,line.strip()))
  
 # print(sentiment.group(0))
 
fileToRead.close()

training_sentences = statementsForTraining
training_labels = sentimentsForTraining

#print(training_sentences[9])
#print(training_labels[9])

#print(len(training_sentences))






In [ ]:
##do the same with testdata
statementsForTesting = []
sentimentsForTesting = []

fileToRead = open(testing_file, 'r')

while True:
  #next line in file
  line = fileToRead.readline()

  if line == "":
   break

  sent = re.search("OTHER|OFFENSE",line)

  line = remove_clutter(line)

    

  statementsForTesting.append(line)
  #print(len(line))
  #sentimentsForTesting.append(sent.group(0))

  if sent.group(0) == "OTHER": 
    sentimentsForTesting.append(0)
  else:
    sentimentsForTesting.append(1)

  if not line:
    break


fileToRead.close()


testing_sentences = statementsForTesting
testing_labels = sentimentsForTesting
print(len(testing_sentences))
#print(testing_sentences)   
#print(testing_labels)

3532


Padding -> im Prinzip die Sätze alle auf die gleiche Länge bringen (in Theorieteil näher drauf eingehen) -> maxlen-Paramter ist für pad_sequences vorhanden, wenn nicht gesetzt -> alles wird auf die Länge des längsten Satzes aufgefüllt -> print(padded(shape)) gibt aus, wie viele Datensätze padded wurden und gibt an, wie viele Tokens der längste Datensatz hat
print(padded[x]) gibt einen Satz aus (bereits tokenisiert)

Padding -> sowohl für Trainigs- als auch für Testdaten

Und: Daten als numpy-Arrays speichern (notwendig für tensorflow).

 -> Tokenizer https://towardsdatascience.com/text-classification-in-keras-part-2-how-to-use-the-keras-tokenizer-word-representations-fd571674df23 

In [ ]:
tokenizer = Tokenizer(oov_token="OOV")
tokenizer.fit_on_texts(training_sentences)

#creating a word index - nur die Trainigsdaten
word_index = tokenizer.word_index

validation_size = 500


training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded_training = pad_sequences(training_sequences, maxlen=max_length, padding='post')
print(len(padded_training))

validation_sequences = padded_training[0:validation_size]
validation_labels = training_labels[0:validation_size]

padded_training = padded_training[validation_size:]
training_labels = training_labels[validation_size:]

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_testing = pad_sequences(testing_sequences, maxlen=max_length,  padding='post')

#print(validation_sequences[499])
print(padded_training[0])
#print(len(validation_labels))
#print(len(training_labels))


nppadded_training = np.array(padded_training)
nptraining_labels = np.array(training_labels)

nppadded_validation = np.array(validation_sequences)
npvalidation_labels = np.array(validation_labels)

nppadded_testing = np.array(padded_testing)
nptesting_labels = np.array(testing_labels)


print(len(nppadded_training))
print(len(nptraining_labels))
print(len(word_index))

#print(statementsForTraining[2])
#print(nppadded_training[4])
#print(nppadded_training.shape)
#print(nptraining_labels[4])
#print(nppadded_testing.shape)
#print(word_index) 


5009
[  12 3993   11   41 6730 1042    4    5  202   39    3 2930   49 1363
  812  495 3994    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
4509
4509
15130



Dann: Embedding mit entweder word2vec oder glove
Word2vec-Daten sind hier: http://vectors.nlpl.eu/repository/#
Glove -> https://nlp.stanford.edu/projects/glove/ Da mal den Twitter-Vector runtergeladen
Wenn Daten in GoogleDrive sind (wäre auch via url möglich..), muss Drive gemounted werden ( https://buomsoo-kim.github.io/colab/2020/05/09/Colab-mounting-google-drive.md/ ) aber auch hier https://enjoymachinelearning.com/posts/colab-with-google-drive/

Die Word2vec vectors: http://vectors.nlpl.eu/repository/# 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
os.listdir("/content/drive/MyDrive/Colab Notebooks")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['test 2.ipynb',
 'germeval_training.txt',
 'TestNote.ipynb',
 'glove.twitter.27B.50d.txt',
 'glove.twitter.27B.200d.txt',
 'glove.840B.300d.txt',
 'tensorboard.gdoc',
 'keras.gdoc',
 'LSTM.ipynb',
 'CNN_try.ipynb',
 'CNN embedded.ipynb']


Keras-Doc : https://keras.io/examples/nlp/pretrained_word_embeddings/#load-pretrained-word-embeddings 
embedding mit initializers.. weights müsste auch funktionieren

Diese erstellte Matrix mit Embedding dann in die Embedding-Schicht einbinden,  Trainable auf False, weil sich die Werte nicht durch Training anpassen sollen

In [ ]:
#Größe Vokabel -> wordindex + 2 (weil padding + OOV) 
hits = 0
misses = 0

vocabulary_size = len(word_index)+2

# dann erstell ich ein Wörterbuch mit Namen "embedding_vector", dort sind dann
#die keys drinnen, die in glove-Datei drinnen sind mit dem entsprechenden Key

embedding_index_glove = {}
f = open('/content/drive/MyDrive/Colab Notebooks/glove.twitter.27B.200d.txt')
for line in f:
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype='float32')
  embedding_index_glove[word] = coef

print("%d gefunden: "% len(embedding_index_glove))

#Dann noch eine Embedding-Matrix erstellen
#zweiter Wert = Embedding-Dimension der Datei, in dem Fall 200

glove_matrix = np.zeros((vocabulary_size,200))
for word, index in tokenizer.word_index.items():
    embedding_value = embedding_index_glove.get(word)
    if embedding_value is not None:
      glove_matrix[index] = embedding_value
      hits+=1
    else:
      misses+=1

print("hits %d and %d misses"%(hits,misses))


1193514 gefunden: 
hits 6747 and 8383 misses


In [ ]:
from tensorflow.keras.layers import Embedding
model = tf.keras.Sequential()
#Embedding -> hier dann auf das embedding verweisen, Input_dim -> die Anzahl der Wörter im word_index, output -> in dem Fall passend zum verwendeten Vektor
#input-length -> auf 60  gepadded, trainable -> nein, weil nichts verändert werden soll
#embeddings_initializer=keras.initializers.Constant(glove_matrix) vs weights = [glove_matrix]
#Convolutional layers => filters = neurons.. mit 100? 265?
model.add(tf.keras.layers.Embedding(vocabulary_size, output_dim=200, input_length=60, embeddings_initializer = keras.initializers.Constant(glove_matrix), trainable= False))
model.add(tf.keras.layers.Dropout(0.4))
#filters davor: waren 125..
#model.add(tf.keras.layers.Conv1D(filters=25, kernel_size=5, padding="valid", activation="relu"))
#model.add(tf.keras.layers.MaxPooling1D())
model.add(tf.keras.layers.Conv1D(filters=25, kernel_size=5, padding="valid", activation="relu"))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(70, activation="relu"))
#model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

#nppadded_training = np.asmatrix(nppadded_training) not necessary
#nppadded_testing = np.asmatrix(nppadded_testing)





Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 200)           3026400   
_________________________________________________________________
dropout (Dropout)            (None, 60, 200)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 56, 25)            25025     
_________________________________________________________________
global_average_pooling1d (Gl (None, 25)                0         
_________________________________________________________________
dense (Dense)                (None, 70)                1820      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 71        
Total params: 3,053,316
Trainable params: 26,916
Non-trainable params: 3,026,400
_________________________________________

In [ ]:
model.layers[2].get_weights()[0].shape
#weights=np.random.rand(5,200,100)
#bias=np.random.rand(100)
#model.layers[2].set_weights([weights, bias])

#model.layers[2].get_weights()[0]
#https://androidkt.com/set-custom-weights-keras-using-numpy-array/

(5, 200, 25)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks

In [ ]:
%load_ext tensorboard
#nur einmal pro Sitzung notwendig



In [ ]:

logs_base_dir = "./logs"
callbackForTB = tf.keras.callbacks.TensorBoard(logs_base_dir)


In [ ]:
#Werte der Convolutional-Schichten variieren (Filter), Batch-Size variieren, kernel-size.. Evtl andere Aktivierungsfunktion?
#callbacks=tf.keras.callbacks.TensorBoard(logs_base_dir), epochs 9, evtl weniger?
model.fit(nppadded_training, nptraining_labels, batch_size=30, validation_data=(nppadded_testing, nptesting_labels), epochs=8, callbacks=[callbackForTB])
#

In [ ]:
%tensorboard --logdir {logs_base_dir}

In [ ]:
results = model.evaluate(nppadded_testing, nptesting_labels, batch_size=batch_size)
print("test loss, test acc:",results)



make some predictions of test data and save it to variable. verbose=0 -> do not generate output
Gute Quelle: https://deeplizard.com/learn/video/2f-NjDUvZIE 

In [ ]:
CNN_predictions = model.predict(nppadded_testing[10:110], verbose=0)

look at the predictions made -> two columns: first column: probability of predicting a 0 (not negative), second column: probability of predicting a 1 (negative)

In [ ]:
for p in predictions:
  print(p)

rounded predictions: give prediction value of most likely prediction (0 or 1). Printing the output of rounded prediction shows the prediction made by the model on the data (which output is most likely)

In [ ]:
prediction_rounded = np.argmax(predictions, axis=-1)

for p in prediction_rounded:
  print(p)

Confusion Matrix..
https://deeplizard.com/learn/video/km7pxKy4UHU 

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
confusion = confusion_matrix(y_true=nptesting_labels[10:110], y_pred=prediction_rounded)

.....Below, we have a function called plot_confusion_matrix() that came directly from scikit-learn's website. This is code that they provide in order to plot the confusion matrix. (https://deeplizard.com/learn/video/km7pxKy4UHU)
und code von https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py 

mmh https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix
-> also look at classification report
also, sckit stellt eine plot_confusion_matrix-Funktion zur Verfügung, da brauche ich eigentlich nichts mehr machen, außer sie aufzurufen?
Labels kann ich da auch definieren

In [ ]:
confusion_labels =['neutral','negative']
plot_confusion_matrix(cm=confusion, classes=confusion_labels, title='CNN Confusion Matrix')

F1-Score für jede Epoche https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d

Sehr gute Ressource für die verschiedenen Scores und Metriken https://neptune.ai/blog/evaluation-metrics-binary-classification#10 